<a href="https://colab.research.google.com/github/nabeel-ds/Web-Scraping/blob/main/Scraper_2_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kora -q
from kora.selenium import wd
from bs4 import BeautifulSoup
import csv
import time
import requests

Start_url= "https://exoplanets.nasa.gov/discovery/exoplanet-catalog/"
wd.get(Start_url)
time.sleep(10)

     |████████████████████████████████| 57 kB 2.7 MB/s 
     |████████████████████████████████| 56 kB 3.2 MB/s 


In [ ]:
headers = ["name", "light_years_from_earth", "planet_mass", "stellar_magnitude", "discovery_date", "hyperlink", "planet_type", "planet_radius", "orbital_radius", "orbital_period", "eccentricity"]
planet_data = []
new_planet_data = []
def scrape():
    for i in range(1, 100):
        while True:
            time.sleep(2)
            soup = BeautifulSoup(wd.page_source, "html.parser")
            current_page_num = int(soup.find_all("input", attrs={"class", "page_num"})[0].get("value"))
            if current_page_num < i:
                browser.find_element_by_xpath('//*[@id="primary_column"]/footer/div/div/div/nav/span[2]/a').click()
            elif current_page_num > i:
                browser.find_element_by_xpath('//*[@id="primary_column"]/footer/div/div/div/nav/span[1]/a').click()
            else:
                break
        for ul_tag in soup.find_all("ul", attrs={"class", "exoplanet"}):
            li_tags = ul_tag.find_all("li")
            temp_list = []
            for index, li_tag in enumerate(li_tags):
                if index == 0:
                    temp_list.append(li_tag.find_all("a")[0].contents[0])
                else:
                    try:
                        temp_list.append(li_tag.contents[0])
                    except:
                        temp_list.append("")
            hyperlink_li_tag = li_tags[0]
            temp_list.append("https://exoplanets.nasa.gov"+hyperlink_li_tag.find_all("a", href=True)[0]["href"])
            planet_data.append(temp_list)
        wd.find_element_by_xpath('//*[@id="primary_column"]/footer/div/div/div/nav/span[2]/a').click()
        



In [ ]:
def scrape_more_data(hyperlink):
    try:
        page = requests.get(hyperlink)
        soup = BeautifulSoup(page.content, "html.parser")
        temp_list = []
        for tr_tag in soup.find_all("tr", attrs={"class": "fact_row"}):
            td_tags = tr_tag.find_all("td")
            for td_tag in td_tags:
                try:
                    temp_list.append(td_tag.find_all("div", attrs={"class": "value"})[0].contents[0])
                except:
                    temp_list.append("")
        new_planet_data.append(temp_list)
    except:
        time.sleep(1)


In [ ]:
scrape()

In [ ]:
for index, data in enumerate(planet_data):
    scrape_more_data(data[5])
final=[]
for index, data in enumerate(planet_data):
    new_planet = new_planet_data[index]
    new_planet = [elem.replace("\n", "") for elem in new_planet]
    new_planet = new_planet[:7]
    final.append(data+ new_planet)



In [ ]:
with open("final.csv", "w") as f:
        csvwriter = csv.writer(f)
        csvwriter.writerow(headers)
        csvwriter.writerows(final)




In [ ]:
import pandas as pd
df1= pd.read_csv("/content/final.csv")
df1.head(10)

,,name,light_years_from_earth,planet_mass,stellar_magnitude,discovery_date,hyperlink,planet_type,planet_radius,orbital_radius,orbital_period,eccentricity
11 Comae Berenices b,304.0,19.4 Jupiters,4.72307,2007,https://exoplanets.nasa.gov/exoplanet-catalog/...,Gas Giant,2007,19.4 Jupiters,1.08 x Jupiter,1.29 AU,326 days,0.23
11 Ursae Minoris b,409.0,14.74 Jupiters,5.01300,2009,https://exoplanets.nasa.gov/exoplanet-catalog/...,Gas Giant,2009,14.74 Jupiters,1.09 x Jupiter,1.53 AU,1.4 years,0.08
14 Andromedae b,246.0,4.8 Jupiters,5.23133,2008,https://exoplanets.nasa.gov/exoplanet-catalog/...,Gas Giant,2008,4.8 Jupiters,1.15 x Jupiter,0.83 AU,185.8 days,0.0
14 Herculis b,58.0,4.66 Jupiters,6.61935,2002,https://exoplanets.nasa.gov/exoplanet-catalog/...,Gas Giant,2002,4.66 Jupiters,1.15 x Jupiter,2.93 AU,4.9 years,0.37
16 Cygni B b,69.0,1.78 Jupiters,6.21500,1996,https://exoplanets.nasa.gov/exoplanet-catalog/...,Gas Giant,1996,1.78 Jupiters,1.2 x Jupiter,1.66 AU,2.2 years,0.68
18 Delphini b,249.0,10.3 Jupiters,5.51048,2008,https://exoplanets.nasa.gov/exoplanet-catalog/...,Gas Giant,2008,10.3 Jupiters,1.11 x Jupiter,2.6 AU,2.7 years,0.08
1RXS J160929.1-210524 b,454.0,8 Jupiters,12.61800,2008,https://exoplanets.nasa.gov/exoplanet-catalog/...,Gas Giant,2008,8 Jupiters,1.664 x Jupiter,330.0 AU,6505.9 years,0.0
24 Bootis b,313.0,0.91 Jupiters,5.59000,2018,https://exoplanets.nasa.gov/exoplanet-catalog/...,Gas Giant,2018,0.91 Jupiters,1.24 x Jupiter,0.19 AU,30.4 days,0.04
24 Sextantis b,235.0,1.99 Jupiters,6.45350,2010,https://exoplanets.nasa.gov/exoplanet-catalog/...,Gas Giant,2010,1.99 Jupiters,1.19 x Jupiter,1.333 AU,452.8 days,0.09
24 Sextantis c,235.0,0.86 Jupiters,6.45350,2010,https://exoplanets.nasa.gov/exoplanet-catalog/...,Gas Giant,2010,0.86 Jupiters,1.24 x Jupiter,2.08 AU,2.4 years,0.29
